# <font color=orange><div align="center">SDP Project - Question 1</div></font>

### <font color=orange><div align="center">15/12/2025</div></font>
### <font color=orange><div align="center">Ouissal BOUTOUATOU - Alar TAOUDI - Mohammed SBAIHI</div></font>


Refer to [Problem Formulation](Question1.md) for riguourous mathematical formulation of the problem

In [59]:
# Gurobi module
from gurobipy import *

## Instanciation

In [60]:
CANDIDATES = {
    "x": [85, 81, 71, 69, 75, 81, 88], #Xavier
    "y": [81, 81, 75, 63, 67, 88, 95], #Yovanne
}

WEIGHTS = [8, 7, 7, 6, 6, 5, 6]

In [61]:
def pros_and_cons(x,y):
    """Returns the pros(x,y) and cons(x, y) mappings: Course -> Contribution to x > y
    Inputs: x, y students
    N.B: courses are annotated using integers instead of alphabets
    """
    assert x in CANDIDATES, f"{x} isn't in candidates"
    assert y in CANDIDATES, f"{y} isn't in candidates"

    pros = {
        i : (CANDIDATES[x][i] - CANDIDATES[y][i])*WEIGHTS[i]
        for i in range(len(CANDIDATES[x])) 
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) > 0
    }

    cons = {
        i : (CANDIDATES[x][i] - CANDIDATES[y][i])*WEIGHTS[i]
        for i in range(len(CANDIDATES[x])) 
        if (CANDIDATES[x][i] - CANDIDATES[y][i]) < 0
    }
    return pros, cons

In [62]:
PROS, CONS = pros_and_cons("x", "y")
print("The Pros Set:", PROS)
print("The Cons Set:", CONS)

The Pros Set: {0: 32, 3: 36, 4: 48}
The Cons Set: {2: -28, 5: -35, 6: -42}


## Variables

Let $X \in \{0, 1\}^{PROS \times CONS}$ such as:


$$ x_{p, c} = \begin{cases} 1 & \text{if } (p, c) \text{ in the (1-1) explanation} \\ 0 & \text{otherwise} \end{cases} $$

In [63]:
m = Model("Problem_1")
VarDict = {(p, c) : m.addVar(vtype = GRB.BINARY, name=f'x_{p}_{c}')
                    for p in PROS
                    for c in CONS}

display(VarDict)

{(0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 5): <gurobi.Var *Awaiting Model Update*>,
 (0, 6): <gurobi.Var *Awaiting Model Update*>,
 (3, 2): <gurobi.Var *Awaiting Model Update*>,
 (3, 5): <gurobi.Var *Awaiting Model Update*>,
 (3, 6): <gurobi.Var *Awaiting Model Update*>,
 (4, 2): <gurobi.Var *Awaiting Model Update*>,
 (4, 5): <gurobi.Var *Awaiting Model Update*>,
 (4, 6): <gurobi.Var *Awaiting Model Update*>}

## Constraints

1. Every con course should be associated to one and only pro course

    $$ \forall c \in CONS, \sum_{p \in PROS} x_{p, c} = 1 $$


In [64]:
UNICITYCONSTDIC = {
    c: m.addConstr(
        quicksum([VarDict[(p,c)] for p in PROS]) == 1
    )
    for c in CONS
}

2. Every pro course should be assigned to at most one con course

    $$ \forall p \in PROS, \sum_{c \in CONS} x_{p, c} \leq 1 $$


In [65]:
ASSIGNMENTCONSTDIC = {
    p: m.addConstr(
        quicksum([VarDict[(p,c)] for c in CONS]) <= 1
    )
    for p in PROS
}

3. Only valid (1-1) trade-offs should be considered
    $$ \forall p \in PROS, \forall c \in CONS, x_{p, c} w_{p, c} \geq 0 $$

    such that: $w_{p, c}$ is the sum of contribution of courses p and c


In [66]:
TRADEOFFCONSTDIC = {
    (p,c) : m.addConstr(
        VarDict[(p,c)]*(PROS[p] + CONS[c]) >= 0
    )
    for p in PROS 
    for c in CONS
}

### Objective Function

$$ \min  \sum_{p, c} x_{p, c} w_{p, c} $$

N.B: We could actually set the objective function to zero since we are only interested in finding a model that satisfies the constraints. However, we choose to search for a model such that the trade-offs have minimal contribution sums. Although one could argue that a convincing model would be that of trade-offs with maximal contribution sums.

In [67]:
obj = quicksum(
    VarDict[(p,c)]*(PROS[p]+CONS[c])
    for p in PROS
    for c in CONS
)

# obj = 0

m.setObjective(obj, GRB.MINIMIZE)

m.params.outputflag = 0 

m.update()

## Gurobi Optimization

In [87]:
m.optimize()
iteration = 1

if m.status == GRB.INF_OR_UNBD:
    m.setParam(GRB.Param.Presolve, 0)
    m.optimize()

if m.status == GRB.INFEASIBLE:
    print(m.display(), "\n\tTHERE IS NO SOLUTION!!!")
elif m.status == GRB.UNBOUNDED:
    print(m.display(), "\n\tNOT BOUNDED!!!")
else:
    print(f'z* = {round(m.objVal, 2)}'.center(8*14))

print()

                                                   z* = 11.0                                                    



In [93]:
def map_tradeoff(varname: str):
    splits = varname.split("_")
    p = splits[1]
    c = splits[2]
    return (chr(ord(p) + 17), chr(ord(c) + 17))

print("The optimal solution contains the following (1-1) trade-offs:")
for v in m.getVars():
    if v.X == 1:
        print(map_tradeoff(v.VarName))


The optimal solution contains the following (1-1) trade-offs:
('A', 'C')
('D', 'F')
('E', 'G')
